# Chapter 3 - Classification

In [3]:
# Common imports
import numpy as np


# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [5]:
# from sklearn.datasets import fetch_openml
# mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')

C:\Users\Segun Lawal\anaconda3\envs\zero_to_mastery_ml\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [9]:
# Save mnist to datasets folder
import pickle

with open("../datasets/mnist/mnist_784/mnist_784", 'wb') as file_pi:
    pickle.dump(mnist, file_pi)

In [7]:
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [8]:
X, y = mnist["data"], mnist["target"]
X.shape

(70000, 784)